In [1]:
from datetime import datetime
from portfolio.equity_portfolio import EquityPortfolio

In [2]:
start = datetime(2015,1,1)
end = datetime(2021,1,1)

In [3]:
ep = EquityPortfolio(start,end,"test_portfolio")

In [4]:
ep.load()

100%|█████████████████████████████████████████████████████████████████████████████████| 2/2 [00:26<00:00, 13.29s/it]


In [6]:
ep.sim()

 13%|██████████▎                                                                   | 66/501 [00:15<01:47,  4.04it/s]


 27%|████████████████████▌                                                        | 134/501 [00:32<01:26,  4.25it/s]


 40%|███████████████████████████████                                              | 202/501 [00:48<01:08,  4.37it/s]


 54%|█████████████████████████████████████████▍                                   | 270/501 [01:04<00:57,  4.00it/s]


 67%|███████████████████████████████████████████████████▉                         | 338/501 [01:21<00:40,  4.06it/s]


 81%|██████████████████████████████████████████████████████████████▍              | 406/501 [01:37<00:22,  4.18it/s]


 95%|████████████████████████████████████████████████████████████████████████▊    | 474/501 [01:54<00:06,  4.19it/s]


100%|█████████████████████████████████████████████████████████████████████████████████| 2/2 [02:15<00:00, 67.78s/it]
